# Builder Tutorial number 9

The builder tutorials demonstrate how to build an operational GSFLOW model using `pyGSFLOW` from shapefile, DEM, and other common data sources. These tutorials focus on the `gsflow.builder` classes.

## Working with land cover and soil raster data

In this tutorial, we give an overview of how to translate land cover and soils raster data to PRMS parameters. The methods outlined here, use the raster resampling methods outlined in `Builder_tutorial_2`, pyGSFLOW built in builder utilities, and user generated look up tables (remap files) to convert raw data into PRMS parameter values.

In [1]:
import os
import utm
import shapefile
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import flopy
from gsflow.builder import GenerateFishnet, FlowAccumulation
import gsflow

## Applying the methods to the Sagehen 50m example problem

In this example the methods are applied directly to the Sagehen 50m model as they are presented.


Let's start by loading the Modflow model, PRMS parameter file, and the flow direction array that was produced in earlier builder tutorials

In [2]:
# define the input and output data paths
input_ws = os.path.join("data", "sagehen", "50m_tutorials")
geospatial_ws = os.path.join("data", "geospatial")
output_ws = os.path.join("data", "temp")

# Set modflow model and the prms parameter file paths
modflow_nam = "sagehen_50m.nam"
parameter_file = os.path.join(input_ws, "sagehen_50m_initial.param")

# set the flow direction array path
fdir_file = os.path.join(input_ws, "sagehen_50m_flowdir.txt")

Load the Modflow model, PRMS parameter file, and flow direction array

In [3]:
ml = gsflow.modflow.Modflow.load(modflow_nam, model_ws=input_ws)
parameters = gsflow.prms.PrmsParameters.load_from_file(parameter_file)

# check the modelgrid coordinate information to make sure it loaded properly
print(ml.modelgrid.xoffset, ml.modelgrid.yoffset)

# load the flow direction array
flow_directions = np.genfromtxt(fdir_file)

   loading iuzfbnd array...
   loading irunbnd array...
   loading vks array...
   loading eps array...
   loading thts array...
stress period 1:
   loading finf array...
stress period 2:
------------------------------------
Reading parameter file : sagehen_50m_initial.param
------------------------------------
214270.0 4366610.0


### Load and resample raster data

Loading and resampling of raster data is performed using the Raster resampling methods outlined in Builder tutorial 2. Because this is a computationally intensive process, code is provided for completeness, however the default behavior of this notebook is to skip this step and load the saved ASCII versions. Change `resample_rasters=False` to `resample_rasters=True` to run the raster sampling.

In [4]:
resample_rasters = False

Raster data for the sagehen 50m model was octained from National Land Cover Database impervious cover data layer (NLCD, 2016), SSURGO 1:24000 inventory of soil and non-soil layers (USDA, 2021), and Landfile existing vegetation layers (Landfile, 2016).

Define the paths to the raster data

In [5]:
# NLCD impervious land cover
impervious_raster = os.path.join(geospatial_ws, "nlcd2011_imp_utm.img")

# ssurgo soil raster data
awc_raster = os.path.join(geospatial_ws, "awc.img")
clay_raster = os.path.join(geospatial_ws, "clay.img")
sand_raster = os.path.join(geospatial_ws, "sand.img")
ksat_raster = os.path.join(geospatial_ws, "ksat.img")

# Landfile vegetation type and cover raster data
veg_type_raster = os.path.join(geospatial_ws, "us_140evt_utm.img")
veg_cov_raster = os.path.join(geospatial_ws, "us_140evc_utm.img")

#### resampling rasters

For a more complete explanation of raster resampling see Builder tutorial 2.

Note by default this notebook does not resample the rasters, code is included for completeness.

In [6]:
# resample impervious land cover raster
if resample_rasters:
    rs_file = os.path.join(output_ws, "impervious_50m.txt")
    raster = flopy.utils.Raster.load(impervious_raster)
    impervious = raster.resample_to_grid(
            ml.modelgrid,
            band=raster.bands[0],
            method="median",
            multithread=True,
            thread_pool=12
        )
    impervious[ml.modelgrid.idomain[0] == 0] = 0
    impervious /= 100
    np.savetxt(rs_file, impervious)

In [7]:
# resample SSURGO soil raster data
if resample_rasters:
    # create a loop to reduce code and clutter
    rs_files = [
        os.path.join(output_ws, "awc_50m.txt"), 
        os.path.join(output_ws, "clay_50m.txt"),
        os.path.join(output_ws, "sand_50m.txt"),
        os.path.join(output_ws, "ksat_50m.txt")
    ]
    
    ssurgo_rasters = [
        awc_raster,
        clay_raster,
        sand_raster,
        ksat_raster
    ]
    
    for ix, raster_file in enumerate(ssurgo_rasters):
        print("Resampling: {}".format(os.path.split(raster_file)[-1]))
        rs_file = rs_files[ix]
        raster = flopy.utils.Raster.load(raster_file)
        array = raster.resample_to_grid(
            ml.modelgrid,
            band=raster.bands[0],
            method="median",
            multithread=True,
            thread_pool=12
        )
        array[ml.modelgrid.idomain[0] == 0] = 0
        array[array == raster.nodatavals[0]] = np.nanmedian(array)
        if "sand" in raster_file or "clay" in raster_file:
            array /= 100
        
        np.savetxt(rs_file, array)

In [8]:
# resample Landfire vegetative type and cover
if resample_rasters:
    # create a loop to reduce code and clutter
    rs_files = [
        os.path.join(output_ws, "veg_type_50m.txt"), 
        os.path.join(output_ws, "veg_cov_50m.txt"),
    ]
    
    landfire_rasters = [
        veg_type_raster,
        veg_cov_raster,
    ]
    
    for ix, raster_file in enumerate(landfire_rasters):
        print("Resampling: {}".format(os.path.split(raster_file)[-1]))
        rs_file = rs_files[ix]
        raster = flopy.utils.Raster.load(raster_file)
        array = raster.resample_to_grid(
            ml.modelgrid,
            band=raster.bands[0],
            method="nearest",
            multithread=True,
            thread_pool=12
        )
        array[ml.modelgrid.idomain[0] == 0] = 0
        array = array.astype(int)
        np.savetxt(rs_file, array, fmt="%d")

### Loading the resampled raster data

Resampled raster data are saved as delimited ascii files and can be easily loaded into numpy arrays.

Define the paths to the resampled data

In [9]:
# impervious coverage
impervious_file = os.path.join(input_ws, "impervious_50m.txt")

# ssurgo soil coverages
awc_file = os.path.join(input_ws, "awc_50m.txt")
clay_file = os.path.join(input_ws, "clay_50m.txt")
sand_file = os.path.join(input_ws, "sand_50m.txt")
ksat_file = os.path.join(input_ws, "ksat_50m.txt")

# vegetative coverages
veg_type_file = os.path.join(input_ws, "veg_type_50m.txt")
veg_cov_file = os.path.join(input_ws, "veg_cov_50m.txt")

Load the data as numpy arrays

In [10]:
# impervious coverage
impervious = np.genfromtxt(impervious_file)

# ssurgo soil coverages
awc = np.genfromtxt(awc_file)
clay = np.genfromtxt(clay_file)
sand = np.genfromtxt(sand_file)
ksat = np.genfromtxt(ksat_file)

# vegetative coverages
veg_type = np.genfromtxt(veg_type_file, dtype=int)
veg_cov = np.genfromtxt(veg_cov_file, dtype=int)

### Working with lookup tables (remap files)

Look up table files are used to remap vegetative coverage data and other parameters to values that are compatible with PRMS. For example in the landfire land coverage type raster "Subalpine Douglas-fir Forest" is represented by the numeric value 3233. 

The PRMS cov_type parameter, however, only has 5 options:
   - 0 = bare soil
   - 1 = grasses
   - 2 = shrubs
   - 3 = trees
   - 4 = coniferous
   
Look up table files, such as the ones included with pygsflow, allow the user to map vegetation data to the PRMS parameter. In this example "Subalpine Douglas-fir Forest" (3233) is remaped to the PRMS cov_type parameter value 4 (confierous).

A number of remap files are included with pyGSFLOW for Landfire vegetative cover data in the pyGSFLOW `examples/data/remap_files` directory. The user will need to build their own remap files for other data types, or edit the existing ones. For other common land coverage data types, we welcome community addions and contributions to create a more robust collection of these files.  

Define the path to the remap files

In [11]:
remap_ws = os.path.join("data", "remaps", "landfire")

# cover type remap file
covtype_remap = os.path.join(remap_ws, "covtype.rmp")

# cover density remap files
covden_sum_remap = os.path.join(remap_ws, "covdensum.rmp")
covden_win_remap = os.path.join(remap_ws, "covdenwin.rmp")

# snow interception remap file
snow_intcp_remap = os.path.join(remap_ws, "snow_intcp.rmp")

# rain interception remap file
srain_intcp_remap = os.path.join(remap_ws, "srain_intcp.rmp")

# rooting depth remap file
root_depth_remap = os.path.join(remap_ws, "rtdepth.rmp")

### Loading the remap files and working with builder_utils

The `builder_utils` module contains a collection of functions that assist in translating raster data to PRMS parameters and pyGSFLOW `ParamerRecord` objects that can be added to an existing `PrmsParameter` object.

Let's start with importint the `builder_utils` module

In [12]:
from gsflow.builder import builder_utils as bu

Now let's load the remap files 

In [13]:
covtype_lut = bu.build_lut(covtype_remap)
covden_sum_lut = bu.build_lut(covden_sum_remap)
covden_win_lut = bu.build_lut(covden_win_remap)
snow_intcp_lut = bu.build_lut(snow_intcp_remap)
srain_intcp_lut = bu.build_lut(srain_intcp_remap)
root_depth_lut = bu.build_lut(root_depth_remap)

The loaded remap files are stored as a python dictionary

In [14]:
type(covtype_lut)

dict

### Building vegetative cover `ParameterRecord` objects

The `builder_utils` module contains a number of functions that can be used to build `ParameterRecord` objects that can then be added to an existing `PrmsParameter` object (and later written to file).

Here we show the included vegetative cover methods

In [15]:
# prms covtype
covtype = bu.covtype(veg_type, covtype_lut)

# covden_sum (summer cover density)
covden_sum = bu.covden_sum(veg_cov, covden_sum_lut)

# covden_win (winter cover density)
covden_win = bu.covden_win(covtype.values, covden_win_lut)

# rad_trncf (short-wave winter radiation coefficient through canopy)
rad_trncf = bu.rad_trncf(covden_win.values)

# snow_intcp (canopy interception coefficient for snow)
snow_intcp = bu.snow_intcp(veg_type, snow_intcp_lut)

# srain_intcp (canopy interception coefficient for summer rain)
srain_intcp = bu.srain_intcp(veg_type, srain_intcp_lut)

# wrain_intcp (canopy interception coefficient for winter rain)
wrain_intcp = bu.wrain_intcp(veg_type, snow_intcp_lut)

print(covtype)
print(type(covtype))


####
cov_type 10
1
nhru
20562
1
0
0
0
0.
.
.
####
<class 'gsflow.prms.prms_parameter.ParameterRecord'>


### Adding vegetative `ParameterRecord` objects to the `PrmsParameters` object

In this section `ParameterRecord` objects are added to the `PrmsParameters` object using the built in `add_record_object` method.

The `add_record_object` method has two parameters:
   - `record_object` : a `ParameterRecord` object
   - `replace` : bool, a flag to replace an existing parameter if it exists. Default is True

In [16]:
parameters.add_record_object(covtype)
parameters.add_record_object(covden_sum)
parameters.add_record_object(covden_win)
parameters.add_record_object(rad_trncf)
parameters.add_record_object(snow_intcp)
parameters.add_record_object(srain_intcp)
parameters.add_record_object(wrain_intcp)

### Building soil `ParameterRecord` objects

The `builder_utils` module contains a number of functions that can be used to build `ParameterRecord` objects that can then be added to an existing `PrmsParameter` object (and later written to file).

Here we show the included soil methods

#### calculate root depth, aspect, and slope

In [17]:
# calculate the root depth from veg_type for use in soil parameters
root_depth = bu.root_depth(veg_type, root_depth_lut)

hru_aspect = bu.d8_to_hru_aspect(flow_directions)
hru_slope = bu.d8_to_hru_slope(
    flow_directions,
    ml.modelgrid.top,
    ml.modelgrid.xcellcenters,
    ml.modelgrid.ycellcenters
)

#### calculate soil parameters

For a full description of the soil parameters used in PRMS please see the PRMS input instruction tables 

In [18]:
cellsize = 50
soil_type = bu.soil_type(clay, sand)

# soil moisture params
soil_moist_max = bu.soil_moist_max(awc, root_depth)
soil_moist_init = bu.soil_moist_init(soil_moist_max.values)

# soil recharge params
soil_rech_max = bu.soil_rech_max(awc, root_depth)
soil_rech_init = bu.soil_rech_init(soil_rech_max.values)

# gravity reservoir routing coefficients
ssr2gw_rate = bu.ssr2gw_rate(ksat, sand, soil_moist_max.values)
ssr2gw_sq = bu.ssr2gw_exp(ml.modelgrid.ncpl)

# slow flow gravity reservoir routing coefficients
slowcoef_lin = bu.slowcoef_lin(ksat, hru_aspect.values, cellsize, cellsize)
slowcoef_sq = bu.slowcoef_sq(
    ksat, hru_aspect.values, sand, soil_moist_max.values, cellsize, cellsize
)

### Adding soil `ParameterRecord` objects to the `PrmsParameters` object

In this section `ParameterRecord` objects are added to the `PrmsParameters` object using the built in `add_record_object` method.

The `add_record_object` method has two parameters:
   - `record_object` : a `ParameterRecord` object
   - `replace` : bool, a flag to replace an existing parameter if it exists. Default is True

In [19]:
parameters.add_record_object(hru_slope)
parameters.add_record_object(hru_aspect)
parameters.add_record_object(soil_type)
parameters.add_record_object(soil_moist_max)
parameters.add_record_object(soil_moist_init)
parameters.add_record_object(soil_rech_max)
parameters.add_record_object(soil_rech_init)
parameters.add_record_object(ssr2gw_rate)
parameters.add_record_object(ssr2gw_sq)
parameters.add_record_object(slowcoef_lin)
parameters.add_record_object(slowcoef_sq)

### Building impervious coverage `ParameterRecord` objects

The `builder_utils` module contains a number of functions that can be used to build `ParameterRecord` objects that can then be added to an existing `PrmsParameter` object (and later written to file).

Here we show the included impervious cover methods

In [20]:
# build the percent impervious parameter
hru_percent_imperv = bu.hru_percent_imperv(impervious)

# build the maximum contributing area parameter
carea_max = bu.carea_max(impervious)

### Adding impervious cover `ParameterRecord` objects to the `PrmsParameters` object

In this section `ParameterRecord` objects are added to the `PrmsParameters` object using the built in `add_record_object` method.

The `add_record_object` method has two parameters:
   - `record_object` : a `ParameterRecord` object
   - `replace` : bool, a flag to replace an existing parameter if it exists. Default is True

In [21]:
parameters.add_record_object(hru_percent_imperv)
parameters.add_record_object(carea_max)

## Write the `PrmsParameters` object to File

Now that the land cover and soil information has been added to the `PrmsParameters` object, it can be saved to file for use in the next tutorial using the `write()` method. 

In [22]:
parameters.write(os.path.join(output_ws, "sagehen_50m_lu_soil.params"))

The next tutorial will cover methods to add PRMS climate information